# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [70]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [71]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy.stats import linregress
import glob 
import os 
import math
from matplotlib.ticker import FuncFormatter
from matplotlib.dates import DateFormatter, MonthLocator


In [72]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
5,5,ushuaia,-54.8000,-68.3000,8.81,81,75,3.09,AR,1666108231
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
8,8,sisimiut,66.9395,-53.6735,2.72,98,100,3.79,GL,1666108235
9,9,saint-pierre,-21.3393,55.4781,22.82,68,0,5.14,RE,1666108236


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

pip install geoviews

In [73]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

from matplotlib.colors import LinearSegmentedColormap
colors  = ['green', 'darkorange', 'firebrick'] 
cm      = LinearSegmentedColormap.from_list("Custom", colors, N=100)
#np.random.randint(0,10,100)

plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.linewidth"] = 1.50
plt.figure(figsize=(100,100))
m3 = city_data_df.hvplot.points(x='Lng', y='Lat', geo = True, tiles = "OSM", marker='circle', 
                                size=city_data_df["Humidity"]/2, frame_height=1000, fontsize=18, 
                                xlabel="Longitude", ylabel="Latitude", alpha=10000000.0, cmap=cm, color=np.random.randint(0,577,577),
                                fill_alpha=15, label="City")
m3

:Overlay
   .Tiles.I     :Tiles   [x,y]
   .Points.City :Points   [Lng,Lat]   (_color,_size)

<Figure size 10000x10000 with 0 Axes>

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [74]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE

#city_data_df = city_data_df[(city_data_df['Max Temp'] > 15) & (city_data_df['Max Temp'] < 45) & (city_data_df['Wind Speed'] < 2.5)]

# Drop any rows with null values
# YOUR CODE HERE
city_data_df.dropna()

# Display sample data
# YOUR CODE HERE
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
...,...,...,...,...,...,...,...,...,...,...
572,572,sovetskiy,61.3614,63.5842,3.93,96,100,3.71,RU,1666108564
573,573,tatarsk,55.2190,75.9828,3.78,89,84,2.63,RU,1666108564
574,574,canchungo,12.0672,-16.0333,35.07,61,67,2.91,GW,1666108565
575,575,iquique,-20.2208,-70.1431,17.02,68,6,3.50,CL,1666108566


### Step 3: Create a new DataFrame called `hotel_df`.

In [75]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE

hotel_df =  city_data_df[['City_ID', 'City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE

hotel_df['Hotel Name'] = ""

# Display sample data
# YOUR CODE HERE
hotel_df

,City_ID,City,Country,Lat,Lng,Humidity,Hotel Name
0,0,faya,SA,18.3851,42.4509,35,
1,1,farsund,NO,58.0948,6.8047,100,
2,2,new norfolk,AU,-42.7826,147.0587,58,
3,3,jamestown,US,42.0970,-79.2353,77,
4,4,lanzhou,CN,36.0564,103.7922,48,
...,...,...,...,...,...,...,...
572,572,sovetskiy,RU,61.3614,63.5842,96,
573,573,tatarsk,RU,55.2190,75.9828,89,
574,574,canchungo,GW,12.0672,-16.0333,61,
575,575,iquique,CL,-20.2208,-70.1431,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [65]:
#from config import geoapify_key
geoapify_key = "eb43efe87ad84d32a9e4cb0f86666dfc"

In [68]:
# Set parameters to search for a hotel
radius  = 100000000
#limit   = 100000000
#filters = f"circle:{radius}"
#bias    = f"proximity:{radius}"
# YOUR CODE HERE
#params = {    "filter":filters,
#    "limit":limit,
#    "bias":bias,
#    "apiKey": geoapify_key
    # YOUR CODE HERE
#}
params = {"categories":"airport",
          "apiKey": geoapify_key,
          "limit":20 }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for city, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    
    lat = hotel_df.loc[city, "Lat"]
    lng = hotel_df.loc[city, "Lng"]
    # YOUR CODE HERE
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    #params["filter"] = f"Lat.restaurant.{lat},{lng}"
    #params["filter"] = f"Lat.restaurant.{lng}"
    #params["bias"]   = f"Lat.restaurant.{lat},{lng}"
    #params["bias"]   = f"Lat.restaurant.{lng}"
    #print(params)
    #params["filter"] = # YOUR CODE HERE
    #params["bias"] = # YOUR CODE HERE
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    #name_address = # YOUR CODE HERE
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    #name_address = # YOUR CODE HERE
    name_address = name_address.json()
    
    print(name_address)
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    #try:
    #    hotel_df.loc[city, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    #except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
    #    hotel_df.loc[city, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    #print(f"{hotel_df.loc[city, 'City']} - nearest hotel: {hotel_df.loc[city, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'FeatureCollection', 'features': []}
{'type': 'Fe

,City,Country,Lat,Lng,Humidity,Hotel Name
19,puerto ayora,EC,-0.7393,-90.3518,82,No hotel found
29,saint-louis,SN,16.3333,-15.0000,24,No hotel found
39,tomatlan,MX,19.9333,-105.2500,71,No hotel found
42,margate,GB,51.3813,1.3862,76,No hotel found
52,businga,CD,3.3333,20.8833,87,No hotel found
...,...,...,...,...,...,...
562,tena,EC,-0.9833,-77.8167,62,No hotel found
563,ilhabela,BR,-23.7781,-45.3581,88,No hotel found
566,nhulunbuy,AU,-12.2333,136.7667,78,No hotel found
567,sangamner,IN,19.5667,74.2167,85,No hotel found


In [77]:
# Define the API parameters
geoapify_key = "eb43efe87ad84d32a9e4cb0f86666dfc"
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [82]:
# Set parameters to search for airport's details
radius = 50000
params = {
    "categories":"airport",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the airport search
print("Starting airport details search")

# Iterate through the types_df DataFrame
for City_ID, row in hotel_df.iterrows():
    
    # Get the city's name
    city = row["City"]
    
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
           
    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
        
    # Convert the API response to JSON format
    response = response.json()
    
    print(response)
    
    # Grab the first airport from the results and store the details in the DataFrame
    try:
        hotel_df.loc[City_ID, "Airport Name"] = response["features"][0]["properties"]["name"]
        #hotel_df.loc[City_ID, "IATA Name"] = response["features"][0]["properties"]["datasource"]["raw"]["iata"]
       #hotel_df.loc[City_ID, "Airport Address"] = response["features"][0]["properties"]["address_line2"]
        #hotel_df.loc[City_ID, "Distance"] = response["features"][0]["properties"]["distance"]
        #hotel_df.loc[City_ID, "Website"] = response["features"][0]["properties"]["datasource"]["raw"]["website"]
    except KeyError as e:
        # If no airport is found, log the error.
        print(f"{e.args[0]} not found for {hotel_df.loc[City_ID, 'Airport Name']}")
        
    # Log the search results
    print(f"nearest airport from {city}: {hotel_df.loc[City_ID, 'Airport Name']}")

# Display sample data
cities_pd

Starting airport details search
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'مطار أبها الدولي', 'country': 'Saudi Arabia', 'country_code': 'sa', 'state': "'Asir Province", 'county': 'Abha', 'city': 'Khamis Mushayt', 'postcode': '62411', 'street': 'طريق الأمير سلطان', 'lon': 42.65376644973918, 'lat': 18.2416611, 'formatted': 'Abha International Airport, طريق الأمير سلطان, Khamis Mushayt 62411, Saudi Arabia', 'address_line1': 'Abha International Airport', 'address_line2': 'طريق الأمير سلطان, Khamis Mushayt 62411, Saudi Arabia', 'categories': ['airport', 'airport.international'], 'details': ['details', 'details.airport', 'details.wiki_and_media'], 'datasource': {'sourcename': 'openstreetmap', 'attribution': '© OpenStreetMap contributors', 'license': 'Open Database Licence', 'url': 'https://www.openstreetmap.org/copyright', 'raw': {'ele': 2090, 'ref': 'AHB', 'iata': 'AHB', 'icao': 'OEAB', 'name': 'مطار أبها الدولي', 'osm_id': 9712421, 'aeroway': 'a

{'type': 'FeatureCollection', 'features': []}


IndexError: list index out of range

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)